In [45]:
import pandas as pd

In [46]:
data_hero = pd.read_csv('hero_data.csv')
data_hero.head() 

,Hero,Win Rate,Pick Rate,Ban Rate,Best Lane,Role,Image
0,Miya,50.97%,2.99%,2.55%,['gold lane'],['Marksman'],https://akmweb.youngjoygame.com/web/svnres/img...
1,Balmond,44.17%,0.65%,0.32%,['jungler'],['Fighter'],https://akmweb.youngjoygame.com/web/svnres/img...
2,Saber,49.11%,1.00%,61.90%,"['jungler', 'roam']",['Assassin'],https://akmweb.youngjoygame.com/web/svnres/img...
3,Alice,50.84%,0.09%,0.16%,"['exp lane', 'jungler']","['Mage', 'Tank']",https://akmweb.youngjoygame.com/web/svnres/img...
4,Nana,47.53%,2.40%,42.23%,['mid lane'],['Mage'],https://akmweb.youngjoygame.com/web/svnres/img...


In [48]:
X = data_hero[['Hero', 'Image','Role','Best Lane']].values
print(X)

[['Miya'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_da894b37bfb5cadb32307f371f31918a.png'
  "['Marksman']" "['gold lane']"]
 ['Balmond'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_4df4ce8528166d6e1cc94480f4f0e910.png'
  "['Fighter']" "['jungler']"]
 ['Saber'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_6bbfac806f29d29f17fea9e98d2d2fee.png'
  "['Assassin']" "['jungler', 'roam']"]
 ['Alice'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_558cf35ad7d5f0cc4f0b4cbdf9498f9f.png'
  "['Mage', 'Tank']" "['exp lane', 'jungler']"]
 ['Nana'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_ee1e0d80d87bb614a0c552ef028f85ce.png'
  "['Mage']" "['mid lane']"]
 ['Tigreal'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_8b30576754be1a4f8bebd09df8d6bec7.png'
  "['Tank']" "['roam']"]
 ['Alucard'
  'https://akmweb.youngjoygame.com/web/svnres/img/mlbb/homepage/100_0f51b6906e08897

In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [50]:
s = Service(r"I:\Downloads\chromedriver-win64\chromedriver.exe")

data = []
driver = webdriver.Chrome(service=s)
url = "https://m.mobilelegends.com/rank"
try:
    driver.get(url)
    dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "mt-2684889"))
    )
    dropdown.click()
    mythical_glory = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='mt-2684883 mt-text' and span[text()='Mythical Glory+']]"))
    )
    mythical_glory.click()
    dropdown_date = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "mt-2684840"))
    )
    dropdown_date.click()

    # Tunggu sampai elemen "Past 7 days" muncul dan klik elemen tersebut
    past_7_days = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='mt-2684833 mt-text' and span[text()='Past 7 days']]"))
    )
    past_7_days.click()
    
    target_div_xpath = "//div[@class='mt-2690764 mt-list-layout scroll-y']"
    div_rank_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.mt-2690764.mt-list-layout.scroll-y'))
    )
    child_divs_rank = div_rank_element.find_elements(By.CSS_SELECTOR, 'div')
    print("Total child elements:", len(child_divs_rank))
    
    while True:
        # Scroll down
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", div_rank_element)
        
        # Wait for more data to load
        time.sleep(2)
        
        # Get the current number of child elements
        child_elements = driver.find_elements(By.XPATH, target_div_xpath + "/*")
        
        # Check if the number of child elements has reached 125
        if len(child_elements) >= 125:
            break
    
    div_rank_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.mt-2690764.mt-list-layout.scroll-y'))
    )
    child_divs_rank = div_rank_element.find_elements(By.CSS_SELECTOR, 'div.mt-2684916.mt-empty')
    print("Total child elements:", len(child_divs_rank))
    data = []
    for child_div in child_divs_rank:
        # Mengambil elemen spesifik untuk child_div saat ini
        span_name_element = child_div.find_element(By.CSS_SELECTOR, 'div.mt-2693412.mt-text span')
        pick_rate_element = child_div.find_element(By.CSS_SELECTOR, 'div.mt-2684925.mt-text span')
        win_rate_element = child_div.find_element(By.CSS_SELECTOR, 'div.mt-2684926.mt-text span')
        ban_rate_element = child_div.find_element(By.CSS_SELECTOR, 'div.mt-2687632.mt-text span')
        list_best_counter_for_element = child_div.find_element(By.CSS_SELECTOR, 'div.mt-2690521.mt-list')
        
        counter = []
        child_div_list_best_counter_for = list_best_counter_for_element.find_elements(By.CSS_SELECTOR, 'div.mt-2690522.mt-image img')
        role = []
        lane = []
        for child in child_div_list_best_counter_for:
            img = child.get_attribute('src')
            for hero in X:
                if hero[1] == img:
                    counter.append(hero[0])
                    role = hero[2]
                    lane = hero[3]
        
        name = span_name_element.text
        pick_rate = pick_rate_element.text
        win_rate = win_rate_element.text
        ban_rate = ban_rate_element.text
        
        print(name, pick_rate, win_rate, ban_rate, counter, role, lane)
        data.append([name, pick_rate, win_rate, ban_rate, counter , role, lane])
    
except Exception as e:
    print("Terjadi kesalahan:", e)

Total child elements: 3
Total child elements: 125
Lolita 0.69% 62.98% 6.75% ['Granger', 'Cyclops', 'Diggie', 'Bane', 'Bruno'] ['Marksman'] ['gold lane']
Freya 0.99% 59.53% 57.75% ['Faramis', 'Diggie', 'Mathilda', 'Balmond', 'Estes'] ['Support'] ['roam']
Zhask 1.94% 59.47% 63.89% ['Lolita', 'Alice', 'Phoveus', 'Jawhead', 'Zhuxin'] ['Mage'] ['mid lane']
Chip 0.66% 57.62% 70.51% ['Baxia', 'Aldous', 'Grock', 'Hayabusa', 'Valentina'] ['Mage'] ['mid lane']
Edith 1.52% 56.71% 2.95% ['Alice', 'Julian', 'Belerick', 'Fredrinn', 'Hayabusa'] ['Assassin'] ['jungler']
Diggie 0.49% 56.06% 13.77% ['Karrie', 'Silvanna', 'Eudora', 'Karina', 'Alpha'] ['Fighter'] ['jungler', 'exp lane']
Zhuxin 0.91% 55.65% 85.99% ['Natalia', 'Lolita', 'Aldous', 'Barats', 'Aulus'] ['Fighter'] ['jungler']
Harith 1.45% 54.87% 5.61% ['Grock', 'Atlas', 'X.Borg', 'Terizla', 'Alpha'] ['Fighter'] ['jungler', 'exp lane']
Ruby 2.85% 54.82% 13.17% ['Joy', 'Sun', 'Hayabusa', 'Gloo', 'Masha'] ['Fighter', 'Tank'] ['exp lane']
Terizla 3

In [51]:
with open('data_hero_best_counter_for.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Hero','Pick Rate', 'Win Rate','Ban Rate', 'Best Counter For', 'Role', 'Best Lane'])
    writer.writerows(data)

print("Data telah disimpan ke data_hero_best_counter_for.csv")

Data telah disimpan ke data_hero_best_counter_for.csv
